<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Spark_RDDs_with_sparkContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Practice of RDDs over the spark**

In [ ]:
# Get the hight rated comedy and drama movie names

In [1]:
# Load the pyspark libraries
import pyspark
from pyspark.sql import SparkSession  # For the RDDs the SparkSession is must
from pyspark.sql.functions import *

In [3]:
# create a SparkSession + sparkContext
sc : SparkContext = SparkSession\
.builder\
.master("local")\
.appName("RDDs over the spark")\
.getOrCreate()\
.sparkContext

In [6]:
# create Rdds
movie=sc.textFile("/content/movies.dat")
rating=sc.textFile("/content/ratings.dat")

In [9]:
# check the smaple data of the movie RDD
movie.take(2)

["1::Toy Story (1995)::Animation|Children's|Comedy",
 "2::Jumanji (1995)::Adventure|Children's|Fantasy"]

In [11]:
# check the smaple daeta of the rating RDD
rating.take(2)

['1::1193::5::978300760', '1::661::3::978302109']

In [12]:
# Remove / filter the deails from the data sets split it
def splitText(myfile):
  return myfile.split("::",4)  # 4 is note no of columns will filter

In [13]:
# Recreate the Rdd with split the text
movie=sc.textFile("/content/movies.dat").map(splitText)
rating=sc.textFile("/content/ratings.dat").map(splitText)

In [17]:
# check the data it filter or not
movie.take(5)


[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"],
 ['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy']]

In [18]:
rating.take(5)

[['1', '1193', '5', '978300760'],
 ['1', '661', '3', '978302109'],
 ['1', '914', '3', '978301968'],
 ['1', '3408', '4', '978300275'],
 ['1', '2355', '5', '978824291']]

In [54]:
# filter the genre from movie Rdd
movie.filter(lambda mn : "Comedy" in mn).take(2)

[['5', 'Father of the Bride Part II (1995)', 'Comedy'],
 ['19', 'Ace Ventura: When Nature Calls (1995)', 'Comedy']]

In [57]:
# Join the 2 Rdd
# Below filter made by me
merge=movie.filter(lambda co: "Comedy" in co).keyBy(lambda k : k[0]).join(rating.keyBy(lambda k : k[1]))

In [58]:
merge.take(2)

[('5',
  (['5', 'Father of the Bride Part II (1995)', 'Comedy'],
   ['26', '5', '5', '978144881'])),
 ('5',
  (['5', 'Father of the Bride Part II (1995)', 'Comedy'],
   ['48', '5', '3', '978013996']))]

In [59]:
# get the result
result=merge.groupBy(lambda gbn: gbn[1][0][1])

In [60]:
# Its make a group of the movie name
result.take(2)

[('Father of the Bride Part II (1995)',
  <pyspark.resultiterable.ResultIterable at 0x7ace6215e610>),
 ('Mr. Wrong (1996)',
  <pyspark.resultiterable.ResultIterable at 0x7ace337e10d0>)]

In [75]:
output=result.map(lambda v:(v[0],len(v[1])))

In [73]:
output.take(2)

[('Father of the Bride Part II (1995)', 296), ('Mr. Wrong (1996)', 60)]

In [82]:
myresult=output.max(lambda x:x[1])

In [120]:
print(myresult)

('Being John Malkovich (1999)', 2241)


**Take the movie with highest rated who has genre DRAMA.**

In [84]:
#load the libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [85]:
# Create a session
spark : SparkContext = SparkSession\
.builder\
.master("locall")\
.appName("Genre-Drama")\
.getOrCreate()\
.sparkContext

In [91]:
# write splt string function and clean the data
def splitString(file):
  return file.split("::",4)

In [92]:
# create Rdd for genre Drama
mrdd=spark.textFile("/content/movies.dat").map(splitString)
rrdd=spark.textFile("/content/ratings.dat").map(splitString)

In [96]:
mrdd.take(2)

[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"],
 ['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"]]

In [94]:
rrdd.take(2)

[['1', '1193', '5', '978300760'], ['1', '661', '3', '978302109']]

In [98]:
# get the drama from move Rdd
filterRdd=mrdd.filter(lambda get:"Drama" in get)
filterRdd.take(2)

[['14', 'Nixon (1995)', 'Drama'], ['26', 'Othello (1995)', 'Drama']]

In [105]:
# join the Rdd for rating
jrdd=filterRdd.keyBy(lambda k:k[0]).join(rrdd.keyBy(lambda k: k[1]))

In [106]:
jrdd.take(5)

[('14', (['14', 'Nixon (1995)', 'Drama'], ['8', '14', '4', '978230756'])),
 ('14', (['14', 'Nixon (1995)', 'Drama'], ['69', '14', '5', '977882128'])),
 ('14', (['14', 'Nixon (1995)', 'Drama'], ['123', '14', '4', '979326055'])),
 ('14', (['14', 'Nixon (1995)', 'Drama'], ['136', '14', '4', '977422645'])),
 ('14', (['14', 'Nixon (1995)', 'Drama'], ['187', '14', '4', '977234430']))]

In [107]:
# retrive the names from the above result
rval=jrdd.groupBy(lambda g:g[1][0][1])

In [108]:
rval.take(2)

[('Othello (1995)', <pyspark.resultiterable.ResultIterable at 0x7ace3c18c550>),
 ('Dangerous Minds (1995)',
  <pyspark.resultiterable.ResultIterable at 0x7ace3c0de910>)]

In [115]:
# get undisplayed values w r t length
result=rval.map(lambda element:(element[0],len(element[1])))
result.take(2)

[('Othello (1995)', 100), ('Dangerous Minds (1995)', 141)]

In [116]:
fres=result.max(lambda res:res[1])

In [118]:
print(fres)

('Shawshank Redemption, The (1994)', 2227)
